<a href="https://colab.research.google.com/github/MrKozelberg/laplacian_perceptron/blob/main/laplacian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculation of the Laplace operator for a multilayer perceptron

In [ ]:
# imports
import numpy as np  # to work with arrays
import matplotlib.pyplot as plt  # to make figures

# PyTorch
import torch
from torch import nn


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [ ]:
def f0(x):
    return torch.tanh(x)
  
def f1(x):
  return 1/torch.cosh(x)**2
  
def f2(x):
  return -2*torch.sinh(x)/torch.cosh(x)**3

In [ ]:
class WaveFunction(nn.Module):

  def __init__(self, N, layersizes):
    self.N = N  # size of the input vector
    self.layersizes = layersizes  # sizes of hidden layers
    self.l = len(self.layersizes)  # number of hidden layers

    super(WaveFunction, self).__init__()
        
    self.linear_tanh_stack = [None] * (self.l + 1) * 2
    for i in range(self.l + 1):
      if i == 0:
        self.linear_tanh_stack[2*i] = nn.Linear(N, self.layersizes[i])
      elif i == self.l:
        self.linear_tanh_stack[2*i] = nn.Linear(self.layersizes[i-1], 1)
      else:
        self.linear_tanh_stack[2*i] = nn.Linear(self.layersizes[i-1],
                                                self.layersizes[i])
      self.linear_tanh_stack[2*i+1] = nn.Tanh()
    
    self.h = [None] * (self.l + 1)  # outputs of the layers

  
  def forward(self, x):
    for i in range(self.l + 1):
      if i == 0:
        self.h[i] = self.linear_tanh_stack[2*i+1](
            self.linear_tanh_stack[2*i](x))
      else:
        self.h[i] = self.linear_tanh_stack[2*i+1](
            self.linear_tanh_stack[2*i](self.h[i-1]))
      
    return self.h[-1]


  def grad(self, x):
    """
    finds the gradient of the wave function at a given point x
    """
    y = self.forward(x)  # computes outputs of the layers #
    gradient = torch.zeros(self.N) #
    for t in range(self.N):
      dh_dx = torch.zeros(self.N) #
      dh_dx[t] = 1 # 

      for i in range(self.l + 1):
        w = self.linear_tanh_stack[2*i].weight
        b = self.linear_tanh_stack[2*i].bias

        if i == 0:
          dh_dx = f1(w @ x + b) * (w @ dh_dx)
        else:
          dh_dx = f1(w @ self.h[i-1] + b) * (w @ dh_dx)      
      
      gradient[t] = dh_dx

    return gradient
  

  def laplac(self, x):
    """
    finds the Laplacian of the wave function at a given point x
    """
    y = self.forward(x)  # computes outputs of the layers
    laplacian = 0.0
    for t in range(self.N):
      dh_dx = torch.zeros(self.N)
      dh_dx[t] = 1

      d2h_dx2 = torch.zeros(self.N)

      for i in range(self.l + 1):
        w = self.linear_tanh_stack[2*i].weight
        b = self.linear_tanh_stack[2*i].bias

        # firstly, the new value of the second derivative computes as it is 
        # used the previous value of the first derivative

        if i == 0:
          d2h_dx2 = f2(w @ x + b) * (w @ dh_dx)**2
        else:
          d2h_dx2 = f2(w @ self.h[i-1] + b) * (w @ dh_dx)**2\
                    + f1(w @ self.h[i-1] + b) * (w @ d2h_dx2)

        if i == 0:
          dh_dx = f1(w @ x + b) * (w @ dh_dx)
        else:
          dh_dx = f1(w @ self.h[i-1] + b) * (w @ dh_dx)
      
      # print(d2h_dx2)
      laplacian += d2h_dx2
    
    return laplacian


## Test

In [ ]:
wf = WaveFunction(5, [5]).to(device)
# wf.linear_tanh_stack

### Definition of the weights and biases

In [ ]:
with torch.no_grad():
  wf.linear_tanh_stack[0].weight[:] =torch.nn.parameter.Parameter(torch.diag(torch.ones(wf.linear_tanh_stack[0].weight.shape[0])))
  wf.linear_tanh_stack[0].bias[:] =torch.nn.parameter.Parameter(torch.zeros(wf.linear_tanh_stack[0].bias.shape))
  wf.linear_tanh_stack[2].weight[:] =torch.nn.parameter.Parameter(torch.diag(torch.ones(wf.linear_tanh_stack[2].weight.shape[0])))
  wf.linear_tanh_stack[2].bias[:] =torch.nn.parameter.Parameter(torch.zeros(wf.linear_tanh_stack[2].bias.shape))

### Test the neural network

In [ ]:
x = torch.arange(5) * 0.99

y_an = torch.tanh(torch.sum(torch.tanh(x)))

y_an, wf(x)

(tensor(0.9988), tensor([0.9988], grad_fn=<TanhBackward0>))

### Test of its gradient

In [ ]:
yx_an = 1/torch.cosh(torch.sum(torch.tanh(x)))**2 * 1/torch.cosh(x)**2

yx_an, wf.grad(x)

(tensor([2.3748e-03, 1.0126e-03, 1.7437e-04, 2.4871e-05, 3.4495e-06]),
 tensor([2.3748e-03, 1.0126e-03, 1.7437e-04, 2.4871e-05, 3.4495e-06],
        grad_fn=<CopySlices>))

### Finaly, we test its laplacian

In [ ]:
yxx_an = torch.sum(-2*yx_an*(torch.tanh(x) + y_an/torch.cosh(x)**2))

yxx_an, wf.laplac(x)

(tensor(-0.0076), tensor([-0.0076], grad_fn=<AddBackward0>))